In [ ]:
key = "..."
dify_token = "..."
meet_url = "https://meet.google.com/dsv-thrt-wfo"
url = "https://us-west-2.recall.ai/api/v1/bot/"

In [ ]:
import requests

# create bot
payload = {
    "transcription_options": {
        "provider": "deepgram",
        "deepgram": {"model": "nova-2", "language": "ja", "smart_format": True, "endpointing": 100},
    },
    "meeting_url": meet_url,
    "bot_name": "ミニオン",
}

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": key,
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

In [ ]:
# delete bot 
del_bot_id = "31ea7540-a66e-4ed5-b643-32473dbf5593"
requests.post(f"https://us-west-2.recall.ai/api/v1/bot/{del_bot_id}/leave_call/", headers=headers)

In [ ]:
bot_id = "7266e571-8f55-407b-a4e9-388924264756"

In [ ]:
response = requests.get(f"https://us-west-2.recall.ai/api/v1/bot/{bot_id}/transcript/", headers=headers)
response.json()

In [ ]:
from collections import namedtuple
from datetime import timedelta

# Define the namedtuple
Subtitle = namedtuple('Subtitle', ['start', 'end', 'speaker', 'sentence'])

def format_timestamp(seconds):
    td = timedelta(seconds=seconds)
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = td.microseconds // 1000
    return f"{hours:02}:{minutes:02}:{seconds:02}.{milliseconds:03}"

def to_subtitle_list(data):
    subtitle_list = []
    
    for entry in data:
        speaker = entry["speaker"]
        words = entry["words"]
        
        # Find the first non-empty text to determine the start time
        start_time = None
        for word in words:
            if word["text"]:
                start_time = word["start_timestamp"]
                break
        
        # Find the last non-empty text to determine the end time
        end_time = None
        for word in reversed(words):
            if word["text"]:
                end_time = word["end_timestamp"]
                break
        
        # Collect non-empty text
        text = " ".join(word["text"] for word in words if word["text"])
        
        if text and start_time is not None and end_time is not None:
            start = format_timestamp(start_time)
            end = format_timestamp(end_time)
            subtitle = Subtitle(start=start, end=end, speaker=speaker, sentence=text)
            subtitle_list.append(subtitle)
    
    return subtitle_list

In [ ]:
def send_translation_request(text):
    payload = {
        "inputs": {"input": text},
        "response_mode": "blocking",
        "user": "translate",
    }
    
    headers = {
        "Authorization": f"Bearer {dify_token}",
        "Content-Type": "application/json",
    }
    
    response = requests.post("https://api.dify.ai/v1/completion-messages", json=payload, headers=headers)
    
    answer = response.json().get("answer", "")
    return answer.strip()

send_translation_request("こんにちは、世界！")

In [ ]:
def get_new_transcript():
    response = requests.get(f"https://us-west-2.recall.ai/api/v1/bot/{bot_id}/transcript/", headers=headers)
    data = response.json()
    return to_subtitle_list(data)


print(get_new_transcript())

In [ ]:
def send_message(message):
    url = f"https://us-west-2.recall.ai/api/v1/bot/{bot_id}/send_chat_message/"
    payload = { "message": message }
    response = requests.post(url, json=payload, headers=headers)
    
def send_translation(subtitle, text=None):
    if text is None:
        return 
    message = f"{subtitle.speaker}: {text} (Original: {subtitle.sentence})"
    return send_message(message)

In [ ]:
def run_transcript_query(query, transcripts):
    text = "\n\n".join(
        f"{transcript.speaker}: {transcript.sentence}" for transcript in transcripts
    )
    text = f"{query}\n\n<transcript>\n{text}\n</transcript>"

    payload = {
        "inputs": {},
        "query": text,
        "response_mode": "blocking",
        "user": "helper",
    }

    headers = {
        "Authorization": f"Bearer ...",
        "Content-Type": "application/json",
    }

    response = requests.post(
        "https://api.dify.ai/v1/chat-messages", json=payload, headers=headers
    )
    data = response.json()
    print(data)
    answer = data.get("answer", "")
    citations = data.get("retriever_resources", [])
    citations = "\n".join(citation["document_name"] for citation in citations)
    return answer.strip(), citations


def run_summarization_query(transcripts):
    query = """Here is a the transcript of a presentation so far. 
    The presentation is about introducing a real time assistant for Japan AI meeting, where it can capable to help user on questions. 
    1. summarize into 3 sentences in point form 
    2. Supply additional context in JAI knowledge for company specific context
    3. Support additional context in field knowledge (technical, sales, business, marketing etc) for general viewer
    """
    return run_transcript_query(query, transcripts)


def run_question_query(question, transcripts):
    query = """Here is a the transcript of a meeting so far, answer the following question."""
    query += "\nquestion: " + question.sentence

    return run_transcript_query(query, transcripts)

In [ ]:
from datetime import datetime
import time

class Assistant:
    def __init__(self):
        self.support_period = timedelta(seconds=30)
        self.trigger_word = "バナナ"
        self.last_support_at = None
        self.seens = set()
        

    def assist_once(self):
        
        new_subtitle = get_new_transcript()
        for s in new_subtitle:
            if s not in self.seens:
                print(s)
                translated = send_translation_request(s.sentence)
                send_translation(s, text=translated)
                if self.trigger_word in s.sentence:
                    print(f"Found trigger word: {self.trigger_word}")
                    result, cite = run_question_query(s, self.seens)
                    if result:
                        send_message(result)
                    if cite:
                        send_message(cite)
            self.seens.add(s)
        now = datetime.now()
        if not self.last_support_at or now - self.last_support_at > self.support_period:
            if len(self.seens):
                print("Supporting")
                result, cite = run_summarization_query(self.seens)
                print(result)
                if result:
                    send_message(result)
                if cite:
                    send_message(cite)
            self.last_support_at = now
                
    def run(self):
        while True:
            print("Assisting")
            self.assist_once()
            time.sleep(1)


In [ ]:
a = Assistant()

In [ ]:
a.run()